In [2]:
import warnings
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import re
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def initialize_df():
    global features, maping_description, insurance_features, ads_features, maping_additional

    features = {
        'name': None,
        'model': None,
        'price': None,
        'color': None,
        'fuel_type': None,
        'origin_car': None,
        'car_license': None,
        'lime_type': None,
        'glass': None,
        'motor_power': None,
        'car_speedometer': None,
        'passengers': None,
        'payment_method': None,
        'displayed': None,
        'ex_owners': None,
        'insurance_third': None,
        'supplementary_body': None,
        'total': None,
        'ads_status': None,
        'ads_start_data': None,
        'ads_end_data': None,
        'alarm_devise':None,
        'air_conditioner':None,
        'CD_recorder':None,
        'sunroof':None,
        'leather_upholstery':None,
        'central_closing':None,
        'magnesium_rims':None,
        'air_cushion':None,
         }
    maping_description = {
        "لون السيارة": 'color',
        "نوع الوقود": 'fuel_type',
        "أصل السيارة": 'origin_car',
        "عداد السيارة": 'car_speedometer',
        "أصحاب سابقون": 'ex_owners',
        "رخصة السيارة": 'car_license',
        "نوع الجير": 'lime_type',
        "الزجاج": 'glass',
        "قوة الماتور": 'motor_power',
        "عدد الركاب": 'passengers',
        "وسيلة الدفع": 'payment_method',
        "معروضة": 'displayed'
    }
    maping_additional={
        'جهاز إنذار': 'alarm_devise',
        'مُكيّف': 'air_conditioner',
        'مسجل CD': 'CD_recorder',
        'فتحة سقف': 'sunroof',
        'فرش جلد': 'leather_upholstery',
        'إغلاق مركزي': 'central_closing',
        'جنطات مغنيسيوم': 'magnesium_rims',
        'وسادة حماية هوائية': 'air_cushion',
    }
    insurance_features = ['insurance_third', 'supplementary_body', 'total']
    ads_features = ['ads_status', 'ads_start_data', 'ads_end_data']

In [4]:
def get_model(model_element):
    for row in model_element:
        subitem = str(row.text).split()
        for item in subitem:
            if(item.isdigit()):
                features['model'] = item
                break

In [5]:
def get_price(price_element):
    price = None
    
    if price_element:
        price_text = str(price_element.text)
        for subitem in price_text.split():
            if subitem.isdigit():
                price = int(subitem)
                break

    if price is not None:
        features['price'] = price


In [6]:
def get_description(description_element):
    if description_element:
        for arabic_feature in maping_description.keys():
            description = description_element.find(text=arabic_feature)
            features[maping_description[arabic_feature]] = \
                None if description is None \
                else description.next_element.get_text()


In [7]:
def get_insurance(insurance_element):

    count = 0
    for row in insurance_element:
        subitem = str(row.text).split()
        for item in subitem:
            if(item.isdigit()):
                features[insurance_features[count]] = item
                count += 1

In [8]:
def get_additional_info(additional_element):
    if additional_element:
        for li in additional_element.find_all('li'):
            feature = li.text.strip()
            if feature in maping_additional:
                features[maping_additional[feature]] = 1

In [9]:
data = pd.DataFrame()

folder_path =Path('C:/data') 

for path in folder_path.glob('*.txt'):
    initialize_df()
    with path.open(encoding='utf-8') as file_path:
        # create BeautifulSoup object for file by html parser
        soup = BeautifulSoup(file_path, "html.parser")

        # Extracting name value to features dictionary
        features['name'] = soup.find('h3', class_=None).text

        # Extracting model value to features dictionary
        get_model(soup.find('h5', class_=None))

        # Extracting price value to features dictionary
        get_price(soup.find('h5', class_='post-price'))

        # Extracting description values to features dictionary
        get_description(soup.find('table', class_='list_ads'))
        
        # Extracting additional values to features dictionary
        additional_info_element = soup.find('td', class_='list-additions')
        if additional_info_element:
            get_additional_info(additional_info_element)
            
        insurance_element = soup.find('div', class_='row')
        if insurance_element:
            insurance_element = insurance_element.findAll('td', attrs={'class': None, 'colspan': None})
            get_insurance(insurance_element)
        
        # Convert the dictionary to series
        sample = pd.Series(features)

        # Concating the sample series to the dataframe
        data = pd.concat((data, sample), axis=1, ignore_index=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_16800\2598652444.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description = description_element.find(text=arabic_feature)
C:\Users\dell\AppData\Local\Temp\ipykernel_16800\2598652444.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  description = description_element.find(text=arabic_feature)


In [10]:
data

,0,1,2,3,4,5,6,7,8,9,...,7767,7768,7769,7770,7771,7772,7773,7774,7775,7776
name,كيا اوبتيما,معرض السيارات,معرض السيارات,كيا سورينتو,هونداي افانتي,معرض السيارات,فيات 127,بيجو 208,بي ام دبليو x3,اوبل كورسا,...,هونداي توسان,سكودا اوكتافيا,كيا بيكانتو,كيا سول,كيا مورننغ,فورد ترانزيت,بيجو بارتنر,كيا سورينتو,معرض السيارات,فولكسفاجن ترانسبورتر
model,2014,None,None,2007,2006,None,1982,2014,2015,2015,...,2014,2014,2018,2012,2015,2002,2018,2017,None,2001
price,100000,None,None,60000,43500,None,5500,54000,205000,65000,...,85000,75000,48000,50000,45000,48000,87000,126000,None,28000
color,أبيض عاجي,None,None,سكني,سكني,None,بيج,فضي,أسود,بترولي,...,أسود,أبيض عاجي,أزرق سماوي,أبيض,فيراني,أبيض,فضي,بترولي,None,أبيض
fuel_type,بنزين,None,None,ديزل,بنزين,None,بنزين,بنزين,ديزل,بنزين,...,ديزل,ديزل,بنزين,ديزل,كهرباء,ديزل,ديزل,ديزل,None,ديزل
origin_car,خصوصي,None,None,خصوصي,خصوصي,None,خصوصي,خصوصي,خصوصي,خصوصي,...,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,عمومي,خصوصي,خصوصي,None,عمومي
car_license,فلسطينية,None,None,فلسطينية,فلسطينية,None,فلسطينية,فلسطينية,فلسطينية,فلسطينية,...,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,None,فلسطينية
lime_type,اوتوماتيك,None,None,نصف اوتوماتيك,اوتوماتيك,None,عادي,اوتوماتيك,اوتوماتيك,عادي,...,اوتوماتيك,عادي,اوتوماتيك,اوتوماتيك,اوتوماتيك,عادي,عادي,اوتوماتيك,None,عادي
glass,الكتروني,None,None,الكتروني,الكتروني,None,يدوي,الكتروني,الكتروني,الكتروني,...,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,None,الكتروني
motor_power,2000,None,None,2500,1600,None,906,1200,2000,1400,...,2000,1600,1000,1600,1000,2400,1600,2200,None,2500


In [11]:
csv_file_path = 'C:\df ml\df.csv'
data.to_csv(csv_file_path, index=False)